In [20]:
import vectorbt as vbt
import pandas as pd 
import numpy as np
from datetime import datetime
from kucoin_candle_spot import SpotDataFetcher
from datetime import datetime, timezone
import pandas_ta as ta

from numba import njit
import plotly.graph_objs as go
from vectorbt.portfolio.enums import Direction, SizeType
import talib



## data collection

In [21]:
symbols = [
    "BTC-USDT",   # Solana
]
timeframe = "1day"
start_time = "2020-10-01 10:00:00"
end_time = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")


# Fetch data for each symbol and store in a dictionary

dataframes = {}
for symbol in symbols:
    try:
        fetcher = SpotDataFetcher(symbol, timeframe, start_time, end_time)
        df = fetcher.fetch_candles_as_df()
        df['symbol'] = symbol
        dataframes[symbol] = df
    except Exception as e:
        print(f"Error fetching data: {e}")

symbols = list(dataframes.keys())



INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...
INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 3, Candles: 1606


In [22]:
symbols = list(dataframes.keys())
symbols

['BTC-USDT']

In [23]:

def momentum_strategy(close, high, low, atr_length, ema_length, ema2_length, vola_multiplier, vol_window, symbol=None):
    # Calculate EMA
    EMA = vbt.IndicatorFactory.from_talib('EMA')
    ema = EMA.run(close, timeperiod=ema_length).real.to_numpy()
    ema2 = EMA.run(close, timeperiod=ema2_length).real.to_numpy()

    # Calculate ATR
    ATR = vbt.IndicatorFactory.from_talib('ATR')
    atr = ATR.run(high, low, close, timeperiod=atr_length).real.to_numpy()

    # Function to calculate rolling max
    def rolling_max(arr, window):
        result = np.full_like(arr, np.nan)
        for i in range(window - 1, len(arr)):
            result[i] = np.max(arr[i - window + 1:i + 1])
        return result

    # Function to compute bearish volatility
    def compute_is_bearish_vol(high, low, atr, window):
        rm = rolling_max(high, window=window)
        return (rm - low) > (atr * vola_multiplier)

    # Calculate conditions
    is_bullish = (close > ema) & (close > ema2)
    is_bearish_vol = compute_is_bearish_vol(high, low, atr, window=vol_window)
    is_caution = is_bullish & (is_bearish_vol | (close < ema))
    signal_buy = is_bullish & ~is_caution

    # Define conditions and corresponding values
    conditions = [signal_buy, is_caution]
    values = [10, 5]

    # Create final signal using np.select
    signal = np.select(conditions, values, default=0)

    return signal.reshape(close.shape)

# Create the indicator factory
momentum_indicator = vbt.IndicatorFactory(
    class_name='MomentumStrategy',
    short_name='momentum',
    input_names=['Close', 'High', 'Low'],
    param_names=['atr_length', 'ema_length', 'ema2_length', 'vola_multiplier', 'vol_window','symbol'],
    output_names=['signal']
).from_apply_func(momentum_strategy)


In [24]:

# Define parameter ranges
atr_length = np.arange(5, 8, 1)  # Range from 5 to 7 with step 1
ema_length = np.arange(100, 241, 20)  # Range from 100 to 240 with step 20
ema2_length = np.arange(10, 50, 5)  # Range from 10 to 45 with step 5
vola_multiplier = [1.6, 1.7, 1.8]  # Specific values for volatility multiplier
vol_window = np.arange(7, 10, 1)  # Range from 7 to 9 with step 1

# # # Define parameter ranges
# atr_length = 7# Range from 5 to 7 with step 1
# ema_length = 200  # Range from 100 to 240 with step 20
# ema2_length = 50  # Range from 10 to 45 with step 5
# vola_multiplier = 1.5 # Specific values for volatility multiplier
# vol_window = np.arange(7, 10, 1)  # Range from 7 to 9 with step 1

indicator_obj = {}

for symbol in symbols:
    # Run the indicator
    df_pair = dataframes[symbol]

    indicator = momentum_indicator.run(
        df_pair['close'], df_pair['high'], df_pair['low'],
        atr_length=atr_length,
        ema_length=ema_length,
        ema2_length=ema2_length,
        vola_multiplier=vola_multiplier,
        vol_window=vol_window,
        symbol= symbol,
        param_product=True
    )
    indicator_obj[symbol] = indicator

indicator_obj

{'BTC-USDT': <vectorbt.indicators.factory.MomentumStrategy at 0x13334c1f0>}

In [25]:
def copy_rename_close_series(df_original, df_indicator_signals):

    def repeat_series_horizontally(series, target_shape):
        return np.tile(series.values.reshape(-1, 1), target_shape[1])

    # Assuming df and df_indicator_signals are already defined
    nedded_copies = len(df_indicator_signals.columns)

    # Repeat the 'close' series horizontally to match the shape of df_indicator_signals
    df_dublicate_close = pd.DataFrame(repeat_series_horizontally(df_original['close'], df_indicator_signals.shape), index=df_original.index)

    # Rename columns if shapes match
    # if df_indicator_signals.shape == df_dublicate_close.shape:
    df_dublicate_close.columns = df_indicator_signals.columns
    #     print("Columns have been renamed")
    # else:
    #     print("The DataFrames do not have the same shape.")
    
    
    return df_dublicate_close



df_dublicate_close_pairs ={}
for symbol in symbols:
    print(symbol)
    df = dataframes[symbol]
    df_dublicate_close = copy_rename_close_series(df, indicator_obj[symbol].signal)
    print('New Schape',df_dublicate_close.shape)
    df_dublicate_close_pairs[symbol] = df_dublicate_close
    # df_dublicate_close_pairs.append(df_dublicate_close)

print(df_dublicate_close_pairs.keys())


# here ########
# add choosing cutoff time for backtesting
# create seperation of data for backtesting and validation 



BTC-USDT
New Schape (1606, 1728)
dict_keys(['BTC-USDT'])


In [38]:
df_dublicate_close_pairs

{'BTC-USDT': momentum_atr_length              5                                      \
 momentum_ema_length            100                                       
 momentum_ema2_length            10                                       
 momentum_vola_multiplier       1.6                        1.7            
 momentum_vol_window              7        8        9        7        8   
 momentum_symbol           BTC-USDT BTC-USDT BTC-USDT BTC-USDT BTC-USDT   
 timestamp                                                                
 2020-10-02 00:00:00+00:00  10566.5  10566.5  10566.5  10566.5  10566.5   
 2020-10-03 00:00:00+00:00  10539.0  10539.0  10539.0  10539.0  10539.0   
 2020-10-04 00:00:00+00:00  10664.1  10664.1  10664.1  10664.1  10664.1   
 2020-10-05 00:00:00+00:00  10792.4  10792.4  10792.4  10792.4  10792.4   
 2020-10-06 00:00:00+00:00  10570.3  10570.3  10570.3  10570.3  10570.3   
 ...                            ...      ...      ...      ...      ...   
 2025-02-19 0

In [26]:



missing_data_pairs = []
for symbol in symbols:
    df_check = dataframes[symbol]
    # Create a complete date range based on the start and end of your DataFrame
    complete_date_range = pd.date_range(start=df_check.index.min(), end=df_check.index.max(), freq='1D')

    # Check for missing dates
    missing_dates = complete_date_range.difference(df_check.index)

    if not missing_dates.empty:
        # print('missing data in:',symbol)
        # print("Missing dates:")
        # print(missing_dates)
        missing_data_pairs.append(symbol)
    # else:
         # print("All dates are accounted for.")
    
missing_data_pairs


[]

In [27]:
# df_dublicate_close_pairs['SOL-USDT']

In [28]:
# Numba-compiled order function
@njit
def order_func_nb(c, high, low, open_, entries, sl_prices, tp_prices,tp_hit,entry_price,atr_values):
    close_price = c.close[c.i, c.col]

    # ====================== Active Position ======================
    if (c.position_now > 0):

        # ------------------------- Exit Position -------------------------

        if open_[c.i] <= sl_prices[c.i]:
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,  # Close Entire position

                price=open_[c.i],  

                size_type=SizeType.Amount,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002)


            return value
        
        # ------------------------- Take profit ----------------------
        
        # # if (tp_prices[c.i] < high[c.i]) & (tp_hit[c.i] == False):
        # if (high[c.i] >= tp_prices[c.i]) and (not tp_hit[c.i]):

        #     tp_hit[:] = True

        #     value_sell = vbt.portfolio.nb.order_nb(
        #         size= -0.5,  # Close position

        #         price=tp_prices[c.i],  # take profit at tp price 

        #         size_type=SizeType.Percent,
        #         direction=Direction.LongOnly,

        #         fees=0.001,
        #         slippage=0.002)
            
        #     return value_sell
        
        
        # ------------------------- Trailing SL Update ----------------------

        # sl update for long position
        if (entries[c.i-1, c.col] == 5):  
            highest_low = np.max(low[c.i-7:c.i])
            update2 = highest_low - atr_values[c.i] * 0.2
            if update2 > sl_prices[c.i]:
                sl_prices[:]= update2

        #losen sl for less volatility 
        if (entries[c.i-1, c.col] == 10): 
            highest_low = np.max(low[c.i-7:c.i])
            update = highest_low - atr_values[c.i]
            if update > sl_prices[c.i]:
                sl_prices[:]= update



    
    # ====================== Entry Position ======================
  

    # if not in position search for position to enter
    elif (c.position_now == 0) & (c.i != 0):
        if entries[c.i-1, c.col] == 10:

            # reset values to default 
            sl_prices[:] = np.nan
            entry_price[:] = open_[c.i]
            tp_hit[:] = False

            order = vbt.portfolio.nb.order_nb(
                size=1,  # Adjusted order size

                price=open_[c.i],  # Current closing price
                
                size_type=SizeType.Percent,  # Specify size type
                direction=Direction.LongOnly,  # Long-only trading
                fees=0.001,  # No fees
                slippage=0.002,  # No slippage
                allow_partial=False,  # Do not allow partial fills
                raise_reject=True  # Raise an error if the order is rejected
            )

            # set intial stop loss
            sl_prices[:] = low[c.i] - atr_values[c.i] 

            # set take profit price
            atr_multiple = 1
            tp_prices[:] = entry_price[:] + (atr_values[c.i] * atr_multiple)

            return order

    
    return vbt.portfolio.enums.NoOrder


In [29]:
pf_pairs = {}

for symbol in symbols:
    # candle data
    df = dataframes[symbol]
    close = df_dublicate_close_pairs[symbol]
    open_ = df['open'].to_numpy().flatten()
    high = df['high'].to_numpy().flatten()
    low = df['low'].to_numpy().flatten()

    entries = indicator_obj[symbol].signal.values

    atr_values = talib.ATR(df['high'], df['low'], df['close'], timeperiod=7).to_numpy().flatten()

    # Create an array to store SL prices
    sl_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
    tp_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
    entry_price = np.full(close.shape[0], np.nan)  # Use a 1D array
    tp_hit = np.full(close.shape[0], False)  # Use a 1D array


    # Create portfolio
    pf = vbt.Portfolio.from_order_func(
        close,           # Price DataFrame
        order_func_nb,
        high,
        low,
        open_,
        entries,    # Order function
        sl_prices,
        tp_prices,
        tp_hit,
        entry_price,
        atr_values,  # Pass the SL prices array
        init_cash=500  # Initial cash balance
    )

    pf_pairs[symbol] = pf


In [30]:
# indicator_obj['SOL-USDT'].signal.columns



In [31]:
# Combine the total return and max drawdown for each symbol
first_glance = []

for symbol in symbols:
    # if symbol not in missing_data_pairs:
    total_return = pf_pairs[symbol].total_return()
    max_dd = pf_pairs[symbol].max_drawdown()
    sharpe_ratio = pf_pairs[symbol].sharpe_ratio()
    sortino_ratio = pf_pairs[symbol].sortino_ratio()
    beta = pf_pairs[symbol].beta()
    profit_factor = pf_pairs[symbol].trades.profit_factor()
    win_rate = pf_pairs[symbol].trades.win_rate()
    total_trades = pf_pairs[symbol].trades.count()
        
        # return_and_maxdd = pd.concat([total_return, max_dd], axis=1)
        # return_and_maxdd = pd.concat([total_return, max_dd,total_trades,win_rate,profit_factor], axis=1)

    return_and_maxdd = pd.concat([total_return, max_dd,total_trades,sharpe_ratio,sortino_ratio,win_rate,beta,profit_factor], axis=1)


    first_glance.append(return_and_maxdd)

# Concatenate all DataFrames along the rows
combined_df = pd.concat(first_glance, axis=0)

combined_df

total_return  \
momentum_atr_length momentum_ema_length momentum_ema2_length momentum_vola_multiplier momentum_vol_window momentum_symbol                 
5                   100                 10                   1.6                      7                   BTC-USDT             1.150809   
                                                                                      8                   BTC-USDT             0.730157   
                                                                                      9                   BTC-USDT             0.971862   
                                                             1.7                      7                   BTC-USDT             1.140022   
                                                                                      8                   BTC-USDT             1.036443   
...                                                                                                                                 ...   
7                   240                 45                   1.7                      8                   BTC-USDT             0.955881   
                                                                                      9                   BTC-USDT             0.794349   
                                                             1.8                      7                   BTC-USDT             1.138218   
                                                                                      8                   BTC-USDT             1.105471   
                                                                                      9                   BTC-USDT             0.828406   

                                                                                                                           max_drawdown  \
momentum_atr_length momentum_ema_length momentum_ema2_length momentum_vola_multiplier momentum_vol_window momentum_symbol                 
5                   100                 10                   1.6                      7                   BTC-USDT            -0.319683   
                                                                                      8                   BTC-USDT            -0.326622   
                                                                                      9                   BTC-USDT            -0.333709   
                                                             1.7                      7                   BTC-USDT            -0.348730   
                                                                                      8                   BTC-USDT            -0.343981   
...                                                                                                                                 ...   
7                   240                 45                   1.7                      8                   BTC-USDT            -0.346280   
                                                                                      9                   BTC-USDT            -0.387171   
                                                             1.8                      7                   BTC-USDT            -0.338761   
                                                                                      8                   BTC-USDT            -0.334052   
                                                                                      9                   BTC-USDT            -0.365918   

                                                                                                                           count  \
momentum_atr_length momentum_ema_length momentum_ema2_length momentum_vola_multiplier momentum_vol_window momentum_symbol          
5                   100                 10                   1.6                      7                   BTC-USDT            50   
                                                                                      8                   BTC-U

In [32]:
profit_factor_check = combined_df.sort_values(by=['profit_factor'], ascending=False)
profit_factor_check.head(50)

total_return  \
momentum_atr_length momentum_ema_length momentum_ema2_length momentum_vola_multiplier momentum_vol_window momentum_symbol                 
5                   240                 40                   1.7                      8                   BTC-USDT             1.679424   
                                                             1.6                      8                   BTC-USDT             1.500279   
                                        10                   1.6                      7                   BTC-USDT             1.639227   
                    120                 40                   1.7                      8                   BTC-USDT             2.624553   
                    160                 10                   1.6                      7                   BTC-USDT             1.696796   
6                   240                 40                   1.6                      8                   BTC-USDT             1.267282   
                                                             1.7                      8                   BTC-USDT             1.354344   
7                   240                 40                   1.6                      8                   BTC-USDT             1.298446   
5                   240                 10                   1.7                      7                   BTC-USDT             1.564159   
6                   240                 10                   1.7                      7                   BTC-USDT             1.533196   
5                   240                 40                   1.6                      9                   BTC-USDT             1.343497   
                                        45                   1.7                      8                   BTC-USDT             1.510607   
                    180                 40                   1.7                      8                   BTC-USDT             1.379340   
                    120                 45                   1.7                      8                   BTC-USDT             2.556861   
                    220                 40                   1.7                      8                   BTC-USDT             1.269137   
                    200                 40                   1.7                      8                   BTC-USDT             1.309800   
                    160                 10                   1.7                      7                   BTC-USDT             1.663957   
6                   240                 10                   1.8                      7                   BTC-USDT             1.537030   
5                   120                 35                   1.7                      8                   BTC-USDT             2.408340   
                                        30                   1.7                      8                   BTC-USDT             2.375065   
                                        40                   1.6                      8                   BTC-USDT             2.030928   
                    160                 40                   1.7                      8                   BTC-USDT             1.349918   
6                   160                 10                   1.7                      7                   BTC-USDT             1.588453   
5                   180                 40                   1.6                      8                   BTC-USDT             1.220259   
7                   240                 40                   1.6                      9                   BTC-USDT             1.179737   
5                   240                 10                   1.8                      7                   BTC-USDT             1.437822   
6                   240                 15                   1.7                      9                   BTC-USDT             1.471529   
7                   240                 10                   1.8                      7     

In [39]:
index = 3

profit_factor_check
test_df = profit_factor_check.copy()
test_df=test_df.reset_index()
test_df.iloc[index]
total_return_index = test_df.columns.get_loc('total_return')
total_return_index
name_of_setup_to_investigate = tuple(test_df.iloc[index, :total_return_index ])
name_of_setup_to_investigate

(5, 120, 40, 1.7, 8, 'BTC-USDT')

In [34]:
# index = 1

# combined_df.sort_values(by='total_return', ascending=False).head(50)
# combined_df
# test_df = combined_df.copy()
# test_df=test_df.reset_index()
# test_df.iloc[index]
# total_return_index = test_df.columns.get_loc('total_return')
# total_return_index
# name_of_setup_to_investigate = tuple(test_df.iloc[index, :total_return_index ])
# name_of_setup_to_investigate

In [35]:
name_of_setup_to_investigate[-1]

'BTC-USDT'

In [40]:
# pf_pairs[name_of_setup_to_investigate[-1]][name_of_setup_to_investigate[1:]].plot().show()
pf_pairs[name_of_setup_to_investigate[-1]][name_of_setup_to_investigate].plot().show()

In [37]:
pf_pairs[name_of_setup_to_investigate[-1]][name_of_setup_to_investigate].orders.records_readable


,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,54299,"(5, 240, 40, 1.6, 8, BTC-USDT)",2021-07-29 00:00:00+00:00,0.012458,40096.1322,0.499500,Buy
1,54300,"(5, 240, 40, 1.6, 8, BTC-USDT)",2021-08-04 00:00:00+00:00,0.012458,38125.1968,0.474947,Sell
2,54301,"(5, 240, 40, 1.6, 8, BTC-USDT)",2021-08-07 00:00:00+00:00,0.011042,42925.1790,0.473998,Buy
3,54302,"(5, 240, 40, 1.6, 8, BTC-USDT)",2021-08-27 00:00:00+00:00,0.011042,46735.7412,0.516076,Sell
4,54303,"(5, 240, 40, 1.6, 8, BTC-USDT)",2021-08-29 00:00:00+00:00,0.010513,48989.7840,0.515045,Buy
...,...,...,...,...,...,...,...
84,54383,"(5, 240, 40, 1.6, 8, BTC-USDT)",2025-01-16 00:00:00+00:00,0.013511,100696.5912,1.360470,Buy
85,54384,"(5, 240, 40, 1.6, 8, BTC-USDT)",2025-01-28 00:00:00+00:00,0.013511,101867.4568,1.376289,Sell
86,54385,"(5, 240, 40, 1.6, 8, BTC-USDT)",2025-01-30 00:00:00+00:00,0.013214,103949.4840,1.373540,Buy
87,54386,"(5, 240, 40, 1.6, 8, BTC-USDT)",2025-02-03 00:00:00+00:00,0.013214,97490.6280,1.288195,Sell
